# 앙상블 학습 개요
* 여러 개의 classifier를 생성하고 그 예측을 결합하여 최종 예측을 도출하는 기법
* 정형 데이터 분류 시에 뛰어난 성능
## 앙상블 방식의 유형
### 1. 보팅
* 여러 개의 분류기가 투표를 통해 최종 예측을 결정하는 방식
* 서로 다른 알고리즘을 가진 분류기 결합
### 2. 배깅
* 여러 개의 분류기가 투표를 통해 최종 예측을 결정하는 방식
* 모두 같은 유형의 알고리즘 기반
* 데이터 샘플링을 다양하게 학습
* ex. 랜덤 포레스트 알고리즘
* Bootstrapping: 개별 Classifier에게 데이터를 샘플링해서 추출하는 방식, 중첩 허용
### 3. 부스팅
### 4. 스태킹 앙상블

# 보팅 유형
## Hard Voting
* 에측한 결괏값들 중 다수의 분류기가 결정한 예측값 -> 최종
## Soft Voting
* 분류기들의 결정값의 평균 -> 최종
* 소프트보팅의 예측 성능이 더 좋음

# Voting Classifier
* 주요 생성인자는 estimators, voting값
* estimators: 리스트 값으로 보팅에 사용될 여러 개의 Classifier 객체들을 튜플 형식으로 입력받음
* voting: 'hard'(기본값), 'soft'

In [16]:
import pandas as pd

from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

cancer = load_breast_cancer()

data_df = pd.DataFrame(cancer.data, columns=cancer.feature_names)
data_df.head(3)

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.8,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.9,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.8,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.0,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.5,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758


# Voting Classifier로 유방암 예측

In [24]:
# 개별 모델은 로지스틱 회귀와 KNN
lr_clf = LogisticRegression(solver='liblinear')
knn_clf = KNeighborsClassifier(n_neighbors=8)

# 개별 모델을 소프트 보팅 기반의 앙상블 모델로 구현한 분류기
vo_clf = VotingClassifier(estimators=[('LR', lr_clf), ('KNN', knn_clf)], voting='soft')

X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target,
                                                   test_size=0.2, random_state=156)
# VotingClassifier 학습/예측/평가
vo_clf.fit(X_train, y_train)
pred = vo_clf.predict(X_test)
print('Voting 분류기 정확도: {0:.4f}'.format(accuracy_score(y_test, pred)))

# 개별 모델의 학습/예측/평가
classifiers = [lr_clf, knn_clf]
for classifier in classifiers:
    classifier.fit(X_train, y_train)
    pred = classifier.predict(X_test)
    class_name = classifier.__class__.__name__ # 클래스명을 참조하는 코드임
    print('{0} 정확도: {1:.4f}'.format(class_name, accuracy_score(y_test, pred)))

Voting 분류기 정확도: 0.9561
LogisticRegression 정확도: 0.9474
KNeighborsClassifier 정확도: 0.9386


## 클래스 이름명 참조 방법
1. classifier

이미 생성된 객체(예: 머신러닝 모델 인스턴스, DecisionTreeClassifier() 같은 것).

2. classifier.__class__

해당 객체가 어떤 클래스에서 만들어졌는지 나타내는 클래스 자체를 반환.

예: DecisionTreeClassifier 클래스

3. classifier.__class__.__name__

그 클래스의 이름(문자열)을 반환.

예: "DecisionTreeClassifier"

4. class_name = ...

그 문자열을 class_name 변수에 저장.